# Experiment with Motion Models

- This notebook contains code for experiment with motion models. The purpose of the notebook is to study the error of motion models, which are commonly used in research.
- In this study, we assume that the heading of the user is identical to the heading of the mobile phone and the heading values are computed from quaternions.
- In this study, we explore three step models and three stride-length models, which are commonly used in passive crowd-sourcing methods for WiFi radio map construction.
  - Step Models: local acceleration variance based, angular rate based, auto-correlation based.
  - Stride-Length Models: random, Weinberg, ZUPT

## Import

In [1]:
import sys
import os

# Add paths for resolve 
INDOOR_COMPETITION_20_DIR = os.path.join("..", "indoor-location-competition-20")
sys.path.append(INDOOR_COMPETITION_20_DIR)
CODE_DIR = ".."
sys.path.append(CODE_DIR)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm

from py_indoor_loc.model import read_data_file
from py_indoor_loc.floor_map import read_floor_data, extract_floor_map_geometries, scale
from py_indoor_loc.plot import plot_floor_map
from py_indoor_loc.sensors import compute_earth_acce_heading, compute_earth_acce_heading_ahrs
from py_indoor_loc.pdr.common import compute_step_heading, compute_rel_positions
from py_indoor_loc.pdr.common import RelPositionPredictor, compute_cumulative_step_positions
from py_indoor_loc.pdr.step_detection import AcfSDModel, LocalAcceVarianceSDModel, AngularRateSDModel
from py_indoor_loc.pdr.stride_length import WeinbergSLModel, ZUPTSLModel, RandomSLModel

In [3]:
np.random.seed(19)

In [99]:
def compute_path_length(waypoint):
  return np.linalg.norm(waypoint[1:, 1:] - waypoint[:-1, 1:], axis=1).sum()


def compute_path_duration_secs(waypoint):
  return (waypoint[-1, 0] - waypoint[0, 0]) / 1000

## Experiment Setup

### Track Selection

* In this experiment, we only consider

In [16]:
TRAIN_DATA_DIR = "../../data/train/"

In [28]:
def list_tracks(site_id: str, floor_id: str, data_dir: str) -> list[str]:
  track_floor_path = os.path.join(data_dir, site_id, floor_id)
  return list(glob(track_floor_path + "/*.txt"))

* Compute length and duration of tracks

In [68]:
track_list = []

for site_path in glob(TRAIN_DATA_DIR + "/*"):
  site_id = os.path.basename(site_path)

  for floor_path in glob(site_path + "/*"):
    floor_id = os.path.basename(floor_path)
    
    for track_path in glob(floor_path + "/*.txt"):
      track_id = os.path.basename(track_path)[:-len(".txt")]
      
      track_list.append((site_id, floor_id, track_id))

In [69]:
len(track_list)

26925

In [100]:
track_stats = []

for (site_id, floor_id, track_id) in tqdm(track_list):
  track_path = os.path.join(TRAIN_DATA_DIR, site_id, floor_id, track_id + ".txt")
  try:
    path_data_collection = read_data_file(track_path)
    track_stats.append({
      "site_id": site_id,
      "floor_id": floor_id,
      "track_id": track_id,
      "duration": compute_path_duration_secs(path_data_collection.waypoint),
      "length": compute_path_length(path_data_collection.waypoint),
    })
  
  except Exception as ignored:
    pass

  1%|          | 222/26925 [00:08<16:53, 26.36it/s]


KeyboardInterrupt: 

In [101]:
track_stats_df = pd.DataFrame(track_stats)

In [102]:
track_stats_df.to_csv("../../data/track_stats.csv", index=False)

In [103]:
track_stats_df[track_stats_df["duration"] >= 60]

,site_id,floor_id,track_id,duration,length
13,5a0546857ecc773753327266,B1,5e15731fa280850006f3d013,79.605,48.154400
14,5a0546857ecc773753327266,B1,5e157321a280850006f3d015,78.439,54.540528
33,5a0546857ecc773753327266,B1,5e1580c51506f2000638fc6a,89.977,66.943107
35,5a0546857ecc773753327266,B1,5e1580ccf4c3420006d520e0,107.339,90.026316
36,5a0546857ecc773753327266,B1,5e1580cff4c3420006d520e2,64.965,49.441569
52,5a0546857ecc773753327266,B1,5e15731ca280850006f3d011,88.834,55.475635
96,5a0546857ecc773753327266,B1,5e158f231506f2000638fd3d,77.260,66.685246
105,5a0546857ecc773753327266,B1,5e15be381506f2000638feba,77.009,52.048732
116,5a0546857ecc773753327266,F1,5e15a28d1506f2000638fd5f,72.789,54.645584
122,5a0546857ecc773753327266,F1,5e15a2cd1506f2000638fd6a,70.725,55.351818


### Model Setup

In [81]:
def create_zupt_sl_model(step_detector, **kwargs):
  return ZUPTSLModel(step_detector, **kwargs)

In [82]:
local_acce_var_step_detector = LocalAcceVarianceSDModel(window_size=4, swing_threshold=2, stance_threshold=1, use_ahrs=True)
acf_step_detector = AcfSDModel(t_min=40, t_max=100, use_ahrs=True)
angular_rate_step_detector = AngularRateSDModel(median_filter_window_size=8, stance_threshold=1)

weinberg_sl_model = WeinbergSLModel(window_size=16, use_ahrs=True)
random_sl_model = RandomSLModel(base_sl=1.0, noise_pct=0.15)

zupt_local_acce_var_sl_model = create_zupt_sl_model(local_acce_var_step_detector, window_size=4, fs=50, use_ahrs=True)
zupt_acf_sl_model = create_zupt_sl_model(acf_step_detector, window_size=4, fs=50, use_ahrs=True)
zupt_angular_rate_sl_model = create_zupt_sl_model(angular_rate_step_detector, window_size=4, fs=50, use_ahrs=True)

In [83]:
acf_weinberg = RelPositionPredictor(sd_model=acf_step_detector,
                                    sl_model=weinberg_sl_model,
                                    name="acf + weinberg")

local_acce_var_weinberg = RelPositionPredictor(
    sd_model=local_acce_var_step_detector,
    sl_model=weinberg_sl_model,
    name="local_acce_var + weinberg")

angular_rate_weinberg = RelPositionPredictor(
    sd_model=angular_rate_step_detector,
    sl_model=weinberg_sl_model,
    name="angular_rate + weinberg")

acf_random = RelPositionPredictor(sd_model=acf_step_detector,
                                  sl_model=random_sl_model,
                                  name="acf + random")

local_acce_var_random = RelPositionPredictor(
    sd_model=local_acce_var_step_detector,
    sl_model=random_sl_model,
    name="local_acce_var + random")

angular_rate_random = RelPositionPredictor(sd_model=angular_rate_step_detector,
                                           sl_model=random_sl_model,
                                           name="angular_rate + random")

acf_zupt = RelPositionPredictor(sd_model=zupt_acf_sl_model.step_detection_model,
                                sl_model=zupt_acf_sl_model,
                                name="acf + zupt")

local_acce_var_zupt = RelPositionPredictor(
    sd_model=zupt_local_acce_var_sl_model.step_detection_model,
    sl_model=zupt_local_acce_var_sl_model,
    name="local_acce_var + zupt")

angular_rate_zupt = RelPositionPredictor(
    sd_model=zupt_angular_rate_sl_model.step_detection_model,
    sl_model=zupt_angular_rate_sl_model,
    name="angular_rate + zupt")

models = [
  acf_weinberg,
  local_acce_var_weinberg,
  angular_rate_weinberg,
  acf_random,
  local_acce_var_random,
  angular_rate_random,
  acf_zupt,
  local_acce_var_zupt,
  angular_rate_zupt
]

## Running Experiment

In [90]:
from compute_f import compute_step_positions

In [120]:
def compute_error_metrics(actual_step_positions: np.ndarray, 
                          pred_step_positions: np.ndarray,
                          duration_step_size: int = 10) -> dict:
                          
  actual_path_length = compute_path_length(actual_step_positions[:, 1:])
  predicted_path_length = compute_path_length(pred_step_positions[:, 1:])

  actual_path_rel_timestamps = actual_step_positions[:, 0] - actual_step_positions[0, 0]
  pred_path_rel_timestamps = pred_step_positions[:, 0] - pred_step_positions[0, 0]

  n_segments = np.ceil(actual_path_rel_timestamps[-1] / duration_step_size / 1000).astype(np.int64)

  segments = []
  segment_ae = []
  segment_ape = []
  for k in range(n_segments):
    segments.append((k + 1) * duration_step_size)
    pl_true = compute_path_length(actual_step_positions[actual_path_rel_timestamps < (k + 1) * duration_step_size * 1000, 1:])
    pl_pred = compute_path_length(pred_step_positions[pred_path_rel_timestamps < (k + 1) * duration_step_size * 1000, 1:])
    segment_ae.append(abs(pl_pred - pl_true))
    segment_ape.append(abs(pl_pred - pl_true) / (pl_true + 1e-6))

  return {
    "actual_distance": actual_path_length,
    "predicted_distance": predicted_path_length,
    "distance_error": abs(predicted_path_length - actual_path_length),
    "distance_error_pct": abs((predicted_path_length - actual_path_length) / (actual_path_length + 1e-6)),
    "segments": segments,
    "segment_distance_error": segment_ae,
    "segment_distance_error_pct": segment_ape,
  }

In [125]:
site_floor_tracks = track_stats_df.loc[track_stats_df["duration"] >= 60, ["site_id", "floor_id", "track_id"]].values.tolist()

In [127]:
experiment_results = []

for site_id, floor_id, track_id in tqdm(site_floor_tracks):
  track_path = os.path.join(TRAIN_DATA_DIR, site_id, floor_id, track_id + ".txt")

  try:
    path_data_collection = read_data_file(track_path)
    actual_step_positions = compute_step_positions(path_data_collection.acce, path_data_collection.ahrs, path_data_collection.waypoint)

    for model in models:
      pred_rel_positions = model.predict(path_data_collection)
      pred_step_positions = compute_cumulative_step_positions(pred_rel_positions)
      result = compute_error_metrics(actual_step_positions, pred_step_positions)
      result.update({
        "site_id": site_id,
        "floor_id": floor_id,
        "track_id": track_id,
        "model_name": model.name,
      })
      experiment_results.append(result)

  except Exception as e:
    print(f"Error: {type(e)}: {str(e)}")

100%|██████████| 16/16 [00:49<00:00,  3.09s/it]


In [133]:
pd.DataFrame(experiment_results).groupby("model_name").agg({"distance_error_pct": np.mean})

/tmp/ipykernel_3074366/2737697171.py:1: FutureWarning: The provided callable <function mean at 0x7fc7ed77b420> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  pd.DataFrame(experiment_results).groupby("model_name").agg({"distance_error_pct": np.mean})


,distance_error_pct
model_name,
acf + random,1.084597
acf + weinberg,0.103245
acf + zupt,7.809013
angular_rate + random,0.662566
angular_rate + weinberg,0.831142
angular_rate + zupt,16.537348
local_acce_var + random,0.220422
local_acce_var + weinberg,0.484110
local_acce_var + zupt,32.276845
